In [2]:
%load_ext autoreload
%autoreload 2
import os
import data_prep as dp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import imgaug as ia
import imgaug.augmenters as iaa
import cv2

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

# Modelling

In this notebook I will build a baseline model and then show which different layers can affect them.
Training takes sometime (especially if not using the cloud) so it has been hashed out.

In [3]:
# Set the environment seed for Python
os.environ['PYTHONHASHSEED'] = '0'

seed=101

# Set seed for Numpy
np.random.seed(seed)

## Read in Data

### Train

In [7]:
# Find and label each image according to class
filer=dp.Files()

# Returns list for files
normal = filer.find_files('Docs/all_xrays/train/normal', "0001").files
virus = filer.find_files('Docs/all_xrays/train/virus', "virus").files
bacteria = filer.find_files('Docs/all_xrays/train/bacteria', "bacteria").files

# Create a dictionary for loop and labelling
files_dict = {'normal':normal,'virus':virus,'bacteria':bacteria}
training_dataset = []
iteration=-1

# Loop through lists in dictionary and labels them accordingly
for files in files_dict:
    iteration+=1
    for file in files_dict[files]:
        training_dataset.append((file,iteration))
        
# Turn list into dataframe
df_train = pd.DataFrame(training_dataset, columns=("file","class"))
df_train['class'] = df_train['class'].astype(dtype='category')

# Check populationns and labels
df_train['class'].value_counts() # 0=Normal; 1=Virus; 2=bacteria


2    2196
0    1249
1    1179
Name: class, dtype: int64

### Validation
We need a validation dataset so we can check for overfitting during training as cross-validation requires re-training per k-fold and would be far too time consuming for a neural network.

In [9]:
# Same as for training data
# Find and label each image according to class

normal = filer.find_files('Docs/all_xrays/val/normal', "0001").files
virus = filer.find_files('Docs/all_xrays/val/virus', "virus").files
bacteria = filer.find_files('Docs/all_xrays/val/bacteria', "bacteria").files

files_dict = {'normal':normal,'virus':virus,'bacteria':bacteria}
validation_dataset = []
iteration=-1

# Create list of file and corresponding label
for files in files_dict:
    iteration+=1
    for file in files_dict[files]:
        validation_dataset.append((file,iteration))

# Convert list to dataframe        
df_val = pd.DataFrame(validation_dataset, columns=("file","class"))
df_val['class'] = df_val['class'].astype(dtype='category')

# Check the number of files in each class 
df_val['class'].value_counts() # 0=Normal; 1=Virus; 2=bacteria


2    27
0    15
1    14
Name: class, dtype: int64

In [10]:
directory='Docs/all_xrays/val/'
subfolders=['normal','virus','bacteria']
class_labels=[0,1,2] # Allows changing of problem from binary or multi-class

# Reads the image location and directory, converts data to array and labels the class accordingly
val_data, val_labels = dp.image_data_and_labels('.jpeg',directory,subfolders,class_labels)

print("Total number of validation examples: ", val_data.shape)
print("Total number of labels:", val_labels.shape)

Total number of validation examples:  (56, 224, 224, 3)
Total number of labels: (56, 3)


In [11]:
# Repeat for training set

directory='Docs/all_xrays/train/'
subfolders=['normal','virus','bacteria']
class_labels=[0,1,2]

train_data, train_labels = dp.image_data_and_labels('.jpeg',directory,subfolders,class_labels)

print("Total number of validation examples: ", train_data.shape)
print("Total number of labels:", train_labels.shape)

Total number of validation examples:  (4625, 224, 224, 3)
Total number of labels: (4625, 3)


## Define Augmentations

In [12]:
seq = iaa.OneOf([iaa.Fliplr(0.5),     # Horizontally flip 50% of the images
                    iaa.Affine(rotate=(-20,20)), # Size of rotation range in degrees
                    iaa.Multiply((1.2, 1.5)), # Makes pixels darker or brighter, random amount between 1.2 and 1.5
                    iaa.Crop(percent=(0, 0.05))
                     ]) 

# There is more one can do here - contrast, rotation etc. - but this is enough to demonstrate its purpose.
# It is better to start simple and slowly build up both the data generator and model to avoid overcomplicating things.
# This allows the user to more easily identify what changes produce which results.

## Building the Model
I will be using the VGG16 neural network as this has been pre-trained on ImageNet (dataset of over 14 million images), achieving 92.7% accuracy. It surpasses AlexNet network by replacing large filters of size 11 and 5 in the first and second convolution layers with small size 3x3 filters.

### Basic Model
This will be a baseline model as it has a simple structure with no dropout, batch normalization or separable convolutional. It conists of 2D convolutions and a max pooling layer as well as initial transfer learning from VGG16.

Neural Networks are greedy learners - meaning they often require lots of data to achieve high success. Transfer learning is a useful tool when dealing with smaller data sets such as this as it can provide the neural network with the ability to recognise certain features such as edges and colours.

I will also specify the learning rate of the optimiser, here in this case it is set at 0.001. If our training loss begins to explode (increase rapidly), we will have likely overshot the global minimum and as a result should reduce the learning rate. If we find the loss is not decreasing substantially over several epochs it could be a sign to increase the learning rate.

In [13]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16()

In [14]:
model = Sequential()


# VGG16 first few layers - currently not "frozen".
model.add(VGG16(include_top=False, input_shape=(224,224,3)).layers[0])
model.add(VGG16(include_top=False, input_shape=(224,224,3)).layers[1])
model.add(VGG16(include_top=False, input_shape=(224,224,3)).layers[2])
model.add(VGG16(include_top=False, input_shape=(224,224,3)).layers[3])

# First 2D convolutions (x3) then max pooling. Produces 256 filters.
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name='Conv1_1'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name='Conv1_2'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name='Conv1_3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Pool1'))

# Second 2D convolutions (x3) then max pooling. Produces 512 filters.
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv2_1'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv2_2'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv2_3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Pool2'))

# Thirds 2D convolutions (x3) then max pooling. Produces 512 filters.
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv3_1'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv3_2'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name='Conv3_3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Pool3'))

# Flatten all weights into one layer - this produces the largest number of parameters
# Further reduce the units (number of neurons) before using a final softmax layer to produce our 3 outputs (classes)
model.add(Flatten(name="Flatten"))
model.add(Dense(units=1024, activation="relu", name='Dense1'))
model.add(Dense(units=512, activation="relu", name='Dense2'))
model.add(Dense(units=3, activation="softmax", name='Result'))

In [15]:
# Freezing the initial layers (VGG16) to prevent training and allow transfer learning to our model.

for layer in model.layers[:3]:
    layer.trainable=False

# See the full model architecture, image shape + number of filters and the number of parameters in each layer.
# Fewer parameters reduces training time.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 112, 112, 256)     147712    
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 112, 112, 256)     590080    
_________________________________________________________________
Conv1_3 (Conv2D)             (None, 112, 112, 256)     590080    
_________________________________________________________________
Pool1 (MaxPooling2D)         (None, 56, 56, 256)       0         
__________

In [16]:
# If in local minima while training, Adam optimiser will help get out of local minima and reach global minimum.
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Initial learning rate - affects the rate of gradient descent. Increase if loss does not reduce.
opt = Adam(lr=0.001) 
# Compiles model for training. As we are looking at categorical classes we use categorical cross-entropy loss function.
# Accuracy will be evaluated and displayed at the end of each epoch.
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

checkpoint = ModelCheckpoint("baseline.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

In [22]:
def data_generator(data, batch_size, directory):
    
    n = len(data)
    steps = n//batch_size
    
    batch_images = np.zeros((batch_size, 224,224,3),dtype=np.float32)
    batch_labels = np.zeros((batch_size, 3),dtype=np.float32)
    
    indices = np.arange(n)
    
    # Initialize a counter
    i = 0
    while True:
        np.random.shuffle(indices)
        # Get the next batch 
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['file']
            label = data.iloc[idx]['class']            
            
            # read the image and resize
            if "0001" in img_name:
                img = cv2.imread(str(os.path.join("{}normal".format(directory),str(img_name))))
                img = cv2.resize(img, (224,224)) # Setting the size of all the images as 224x224 - standard input size for VGG-16
            
            elif "virus" in img_name:
                img = cv2.imread(str(os.path.join("{}virus".format(directory),str(img_name))))
                img = cv2.resize(img, (224,224))
                                 
            else:
                img = cv2.imread(str(os.path.join("{}bacteria".format(directory),str(img_name))))
                img = cv2.resize(img, (224,224))
            
            # one hot encoding
            encoded_label = to_categorical(label, num_classes=3)
            
            if img.shape[2]==1:       
                img = np.dstack([img, img, img])  # If grayscale then converts to rgb.
            
            # cv2 reads in BGR mode by default
            orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # normalize the image pixels
            orig_img = img.astype(np.float32)/255.
            
            batch_images[count] = orig_img
            batch_labels[count] = encoded_label
            
            # generating more samples of the undersampled class
            if label==0 and count < batch_size-2:
                aug_img_n = seq.augment_image(img)
                aug_img_n = cv2.cvtColor(aug_img_n, cv2.COLOR_BGR2RGB)
                aug_img_n = aug_img_n.astype(np.float32)/255.

                batch_images[count+1] = aug_img_n
                batch_labels[count+1] = encoded_label
                
                count +=1

            elif label==1 and count < batch_size-2:
                aug_img_v = seq.augment_image(img)
                aug_img_v = cv2.cvtColor(aug_img_v, cv2.COLOR_BGR2RGB)
                aug_img_v = aug_img_v.astype(np.float32)/255.
                
                batch_images[count+2] = aug_img_v
                batch_labels[count+2] = encoded_label

                count +=1
            
            else:
                count+=1
            
            if count==batch_size-1:
                break
            
        i+=1
        yield batch_images, batch_labels
            
        if i>=steps:
            i=0    

In [23]:
batch_size = 50
tf.set_random_seed(101)

# Fit model and begin training. Save the accuracy and loss for each epoch as 'hist' for plotting later
hist = model.fit_generator(steps_per_epoch=10,generator=data_generator(df_train,batch_size,'Docs/all_xrays/train/'),
                           validation_data=data_generator(df_val,batch_size,'Docs/all_xrays/val/'), 
                           validation_steps=10,epochs=40,callbacks=[checkpoint,early])

Epoch 1/40
 1/10 [==>...........................] - ETA: 51:47 - loss: 1.0819 - acc: 0.4000

KeyboardInterrupt: 

In [35]:
# Save the weights of the model as an h5 file. Save only when necessary as files are LARGE! (Github limit=2GB)

# model.save('Baseline_Model_10steps.h5')
# model = load_model('Baseline_Model_1.h5')

In [24]:
# Plot the accuracy and loss for the training and validation steps for every epoch.
# Reveals how much the model is improving and an idea of overfitting 
# If val_loss decreases while training increases - OVERFITTING

plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()
# plt.savefig("Baseline_Model_10steps.png")

NameError: name 'hist' is not defined

From the above plot we can see the loss for both datasets appears to be decreasing while the accuracy for both are also increasing -  apart from some large spikes. We can therefore conclude that we are not overfitting but may be underfitting and could continue training the model for more epochs.